In [1]:
from dotenv import load_dotenv

load_dotenv()


True

## Setup Tools


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from typing import Dict, Any
from tavily import TavilyClient
from langchain.tools import tool

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

In [13]:
#Testing the user of sqlite
 
print(db.get_usable_table_names())
print(query_playlist_db.invoke({"query": "SELECT * FROM Album LIMIT 5;"}))

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
[(1, 'For Those About To Rock We Salute You', 1), (2, 'Balls to the Wall', 2), (3, 'Restless and Wild', 2), (4, 'Let There Be Rock', 1), (5, 'Big Ones', 3)]


## Create State

In [14]:
from langchain.agents import AgentState

class WeddingState(AgentState):
    origin: str
    destination: str
    guest_count: str
    genre: str

## Create Subagents


In [6]:
from langchain.agents import create_agent

# Travel agent
travel_agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
    system_prompt="""
    You are a travel agent. Search for flights to the desired destination wedding location.
    You are not allowed to ask any more follow up questions, you must find the best flight options based on the following criteria:
    - Price (lowest, economy class)
    - Duration (shortest)
    - Date (time of year which you believe is best for a wedding at this location)
    To make things easy, only look for one ticket, one way.
    You may need to make multiple searches to iteratively find the best options.
    You will be given no extra information, only the origin and destination. It is your job to think critically about the best options.
    Once you have found the best options, let the user know your shortlist of options.
    """
)

In [27]:
# Venue agent
venue_agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search],
    system_prompt="""
    You are a venue specialist. Search for venues in the desired location, and with the desired capacity.
    You are not allowed to ask any more follow up questions, you must find the best venue options based on the following criteria:
    - Price (lowest)
    - Capacity (exact match)
    - Reviews (highest)
    You may need to make multiple searches to iteratively find the best options, but to keep things simple, use few searches andonly return a shortlist of 3 venues.
    """
)

In [33]:
# Playlist agent
playlist_agent = create_agent(
    model="gpt-5-nano",
    tools=[query_playlist_db],
    system_prompt="""
    You are a playlist specialist.
    You are not allowed to ask any more follow up questions, Query the sql database and curate the perfect playlist for a wedding given a genre.
    Once you have your playlist, calculate the total duration and cost of the playlist, each song has an associated price.
    If you run into errors when querying the database, try to fix them by making changes to the query.
    Do not come back empty handed, keep trying to query the db until you find a list of songs.
    You may need to make multiple queries to iteratively find the best options.
    """
)

In [ ]:
# Example Query I could pass to the agent to simplify work
# SELECT t.TrackId, t.Name as title, ar.Name as artist, g.Name as genre, (t.Milliseconds/1000.0) as duration_seconds, t.UnitPrice as price FROM Track t JOIN Genre g ON t.GenreId = g.GenreId JOIN Album al ON t.AlbumId = al.AlbumId JOIN Artist ar ON al.ArtistId = ar.ArtistId WHERE g.Name IN ('Pop','Dance','R&B','Rock','Hip-Hop','Soul','Disco') ORDER BY RANDOM() LIMIT 60;

## Main Coordinator


In [15]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def search_flights(runtime: ToolRuntime) -> str:
    """Travel agent searches for flights to the desired destination wedding location."""
    origin = runtime.state["origin"]
    destination = runtime.state["destination"]
    response = await travel_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})
    return response['messages'][-1].content

@tool
def search_venues(runtime: ToolRuntime) -> str:
    """Venue agent chooses the best venue for the given location and capacity."""
    destination = runtime.state["destination"]
    capacity = runtime.state["guest_count"]
    query = f"Find wedding venues in {destination} for {capacity} guests"
    response = venue_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def suggest_playlist(runtime: ToolRuntime) -> str:
    """Playlist agent curates the perfect playlist for the given genre."""
    genre = runtime.state["genre"]
    query = f"Find {genre} tracks for wedding playlist"
    response = playlist_agent.invoke({"messages": [HumanMessage(content=query)]})
    return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
    """Update the state when you know all of the values: origin, destination, guest_count, genre"""
    return Command(update={
        "origin": origin, 
        "destination": destination, 
        "guest_count": guest_count, 
        "genre": genre, 
        "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
        )


In [30]:
from langchain.agents import create_agent

coordinator = create_agent(
    model="gpt-5",
    tools=[search_flights, search_venues, suggest_playlist, update_state],
    state_schema=WeddingState,
    system_prompt="""
    You are a wedding coordinator. Delegate tasks to your specialists for flights, venues and playlists.
    First find all the information you need to update the state. Once that is done you can delegate the tasks.
    Once you have received their answers, coordinate the perfect wedding for me.
    """
)


## Test


In [25]:
# Test of individual agents

from pprint import pprint

print("Test of Travel agent")
response = await travel_agent.ainvoke(
    {
        "messages": [HumanMessage(content="Get me 3 flights from London to Paris")],
    }
)
pprint(response)
response['messages'][-1].content

Test of Travel agent
{'messages': [HumanMessage(content='Get me 3 flights from London to Paris', additional_kwargs={}, response_metadata={}, id='0669f35b-017e-4b42-82bb-85a717dacda9'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1618, 'prompt_tokens': 1367, 'total_tokens': 2985, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1536, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D1O8ex9Jb0QnMHR7EPgf95dDz26lW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bedf0-0147-7ca3-b951-42435f768506-0', tool_calls=[{'name': 'search-flight', 'args': {'flyFrom': 'London', 'flyTo': 'Paris', 'departureDate': '15/05/2026', 'departureDateFlexRange': 0, 'passen

'Here are the top 3 one-way flight options from London to Paris for a 15/05/2026 departure, selected by price (and still sensible if you’re aiming for a quick, direct trip):\n\n| Route (layovers) | Departure → Arrival (local) and duration | Cabin | Price (EUR) | Book link |\n|---|---|---|---:|---|\n| STN → CDG | 15/05/2026 17:00 → 15/05/2026 19:10 (1h 10m) | Economy | 54 | https://on.kiwi.com/UbvwDF |\n| LHR → ORY | 15/05/2026 15:00 → 15/05/2026 17:35 (1h 35m) | Economy | 57 | https://on.kiwi.com/wgk0PC |\n| LTN → CDG | 15/05/2026 06:05 → 15/05/2026 08:30 (1h 25m) | Economy | 90 | https://on.kiwi.com/F4pR4Y |\n\n Shortlist insights:\n- Best price: STN → CDG at 54 EUR (direct). Quickest option among these is also STN → CDG at 1h 10m.\n- Best for a wedding trip vibe: May in Paris is typically pleasant—springtime with mild weather and many outdoor venues.\n- Recommendation: If you want the cheapest and fastest direct option, choose STN → CDG (54 EUR). If you’re flying from central London 

In [28]:
print("Test of search venues agent")
response = venue_agent.invoke(
    {
        "messages": [HumanMessage(content="Find wedding venues in Paris for 100 guests")],
    }
)
pprint(response)
response['messages'][-1].content

Test of search venues agent
{'messages': [HumanMessage(content='Find wedding venues in Paris for 100 guests', additional_kwargs={}, response_metadata={}, id='262ce271-8bbc-4390-983d-c15d5442ebc3'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 532, 'prompt_tokens': 238, 'total_tokens': 770, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D1ODaZUHskg6lLbl8FeRoKPird0MD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bedf4-aa6b-7992-82bd-271834fd56e8-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'Paris wedding venue capacity 100'}, 'id': 'call_MqSmlk2MxjN1Ul9XLB1n1AxW', 'type': 'tool_c

'Here are 3 Paris wedding venues that can host around 100 guests, with high reviews and suitable spaces. Prices vary by date and package, but these are among the top options in the area for 100 guests.\n\n- Hotel Marignan Champs-Elysées\n  - Capacity: up to 100 guests for cocktails/aperitifs (80 for a sit-down meal)\n  - Vibe: Luxury, intimate Parisian setting on the Champs-Élysées\n  - Why it fits: Strong reputation and elegant spaces suitable for cocktail-style receptions or mixed formats\n\n- The Peninsula Paris\n  - Capacity: up to 100 guests for a sit-down dinner in a grand ballroom\n  - Vibe: Ultra-luxury, central location with refined décor and top-tier service\n  - Why it fits: Iconic, highly-rated venue with a dedicated weddings team\n\n- Four Seasons Hotel George V, Paris\n  - Capacity: up to 100 guests for receptions\n  - Vibe: Opulent Art Deco elegance, renowned service and catering\n  - Why it fits: Legendary status and consistently high guest reviews\n\nIf you want, I can

In [34]:
print("Test of playlist agent")
response = playlist_agent.invoke(
    {
        "messages": [HumanMessage(content="Create a playlist for a wedding reception")],
    }
)
pprint(response)
response['messages'][-1].content

Test of playlist agent
{'messages': [HumanMessage(content='Create a playlist for a wedding reception', additional_kwargs={}, response_metadata={}, id='bdd5cecf-e66d-460f-8c51-7c26441f650a'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 708, 'prompt_tokens': 257, 'total_tokens': 965, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D1OHk6L4tT3gtAiZjLUuPZf1waqpC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bedf8-9d4f-7831-b5f6-8b99543b8279-0', tool_calls=[{'name': 'query_playlist_db', 'args': {'query': "SELECT id, title, artist, genre, duration_seconds, price FROM songs WHERE tags LIKE '%wedd

"Here's a wedding-reception friendly playlist pulled from the catalog, balanced for romantic moments and upbeat dancing. I curated 25 tracks across rock, pop, and classic crowd-pleasers with accessible vibes.\n\nCurated Wedding Reception Playlist (25 tracks)\n1) I Heard Love Is Blind — Amy Winehouse — Pop — 2:10 — $0.99 (Track 3470)\n2) Roxanne — The Police — Rock — 3:13 — $0.99 (Track 2650)\n3) Not The Doctor — Alanis Morissette — Rock — 3:48 — $0.99 (Track 48)\n4) Speak To Me/Breathe — Pink Floyd — Rock — 4:42 — $0.99 (Track 2229)\n5) Solitaire — Deep Purple — Rock — 3:26 — $0.99 (Track 824)\n6) Exploder — Audioslave — Rock — 4:23 — $0.99 (Track 93)\n7) Quem Mata A Mulher Mata O Melhor — O Terço — Rock — 3:37 — $0.99 (Track 2021)\n8) Scar Tissue — Red Hot Chili Peppers — Rock — 4:41 — $0.99 (Track 2393)\n9) Cold Turkey — U2 — Pop — 4:31 — $0.99 (Track 3257)\n10) Spellbound — AC/DC — Rock — 2:51 — $0.99 (Track 14)\n11) Sam With The Showing Scalp Flat Top — Frank Zappa & Captain Beefhe

In [36]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
    {
        "messages": [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre")],
    }
)

In [37]:
from pprint import pprint

pprint(response)

{'destination': 'Paris',
 'genre': 'jazz',
 'guest_count': '100',
 'messages': [HumanMessage(content="I'm from London and I'd like a wedding in Paris for 100 guests, jazz-genre", additional_kwargs={}, response_metadata={}, id='c2e468c8-9173-480c-97c8-568c02f7f041'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 549, 'prompt_tokens': 293, 'total_tokens': 842, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 512, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D1OLtgeDWFgSF6O96uMNiMSF4nGYQ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bedfc-893a-7470-87e4-990c7524b1b1-0', tool_calls=[{'name': 'update_state', 'args': {'origin': 'London', 'destination':

In [38]:
print(response["messages"][-1].content)

Wonderful — I’ve got your key details set: London origin, Paris destination, 100 guests, jazz genre. I’ve checked in with my flight, venue, and playlist specialists. Here’s the coordinated plan and what I need from you to lock things in.

Specialist recommendations summary
- Flights (sample direct options, group-friendly):
  - London LTN → Paris CDG, 15 Jun, 19:40–22:00, from about €49 one-way per person (economy).
  - London STN → Paris CDG, 15 Jun, 14:35–17:45, from about €51 one-way per person.
  - London LHR → Paris ORY, 15 May, 15:00–17:35, from about €57 one-way per person.
  - Note: Best practice is to hold a group block across 1–2 flights for flexibility; fares will vary by date and availability. As an alternative, I can also set up a Eurostar group block London–Paris (popular and convenient for 100 guests).
- Venues for 100 guests (shortlist):
  1) Rotonde du Glacier, Palais Garnier
     - Seats up to 100 for dinner, iconic setting in the Opéra district.
     - Premium, breath

link to trace: https://smith.langchain.com/public/7b5fe668-d3e3-4af4-b513-a8cacc0c9e84/r